In [44]:
# Previously created environment in Anaconda console with conda create -n mlops-zoomcamp-project python=3.8
!conda activate mlops-zoomcamp-project


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [45]:
# Sanity check that the environment is active
import os
print (os.environ['CONDA_DEFAULT_ENV'])

mlops-zoomcamp-project


In [46]:
# Question 1: MLflow version

!mlflow --version

mlflow, version 1.28.0


In [47]:
%cd mlflow

/home/usuario/mlops-zoomcamp-project/train_mlflow_prefect/mlflow


In [50]:
!dir

constants.py  preprocess_data.py  register_model.py
hpo.py	      __pycache__	  train.py


In [51]:
!python preprocess_data.py --raw_data_path ../data --dest_path ../output

In [52]:
%cd ../output

/home/usuario/mlops-zoomcamp-project/train_mlflow_prefect/output


In [53]:
#Question 2: How many files saved to output folder?

import os
print(len([name for name in os.listdir('.') if os.path.isfile(name)]))

4


In [54]:
%cd ..

/home/usuario/mlops-zoomcamp-project/train_mlflow_prefect


In [55]:
# Added this in train.py right before rf = RandomForestRegressor(max_depth=10, random_state=0)
#    mlflow.set_tracking_uri("sqlite:///mlflow.db")
#    mlflow.set_experiment("nyc-taxi-experiment")
#    mlflow.sklearn.autolog()
#    with mlflow.start_run():

!python mlflow/train.py

YES
2022/09/12 03:07:11 INFO mlflow.tracking.fluent: Experiment with name 'house-rent-experiment' does not exist. Creating a new experiment.
2022/09/12 03:07:19 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/usuario/anaconda3/envs/mlops-zoomcamp-project/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [56]:
# Question 3: How many parameters are automatically logged by MLflow?

# I executed mlflow ui --backend-store-uri sqlite:///mlflow.db to open mlflow in http://127.0.0.1:5000 and
# checked the parameters of the last run, which were 17

In [57]:
# Question 4: In addition to backend-store-uri, what else do you need to pass to properly configure the server?

# I run in Anaconda console: mlflow ui --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns

In [58]:
# I enclosed the RandomForestRegressor with: with mlflow.start_run():
# and logged the parameters as well as the rmse metric at the last line of the block

!python mlflow/hpo.py

2022/09/12 03:08:53 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
100%|██████████| 50/50 [02:12<00:00,  2.64s/trial, best loss: 23006.91765237763]


In [59]:
# Question 5: What's the best validation RMSE that you got?

# 6.628

In [60]:
!python mlflow/register_model.py

2022/09/12 03:11:30 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
2022/09/12 03:11:38 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/usuario/anaconda3/envs/mlops-zoomcamp-project/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
Successfully registered model 'house-rent-regressor'.
2022/09/12 03:12:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: house-rent-regressor, version 1
Created version '1' of model 'house-rent-regressor'.


In [67]:
# Retrieve the run id associated with the registered best model

import mlflow
from mlflow.tracking import MlflowClient
TRACKING_SERVER_HOST = os.getenv('TRACKING_SERVER_HOST', '')
if TRACKING_SERVER_HOST != '':    
    mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")
else:
    mlflow.set_tracking_uri("http://127.0.0.1:5000")

#mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("house-rent-experiment")

client = MlflowClient()
mlflow_model = client.get_latest_versions("house-rent-regressor", stages=["None"])[0]
run_id_best_model = mlflow_model.run_id
print(run_id_best_model)

8c5e6a3dd6be4339aa29d4f4647ae707


In [68]:
# Question 6: What is the test RMSE of the best model?

mlflow.get_run(run_id_best_model).data.metrics['test_rmse']

24589.048122349584

In [69]:
model_version = mlflow_model.version
new_stage = "Staging"
client.transition_model_version_stage(
    name="house-rent-regressor",
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name="house-rent-regressor",
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1662963175226, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2022-09-12', last_updated_timestamp=1662963879351, name='house-rent-regressor', run_id='8c5e6a3dd6be4339aa29d4f4647ae707', run_link='', source='s3://mlops-zoomcamp-project-mlflow-artifacts/3/8c5e6a3dd6be4339aa29d4f4647ae707/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>